In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import math
from solid import *
from solid import scad_render_to_file, sphere, translate, cylinder
from solid.utils import *
from stl import mesh
import numpy as np
import subprocess
import pdb
import mingus.core.chords as chords
from mingus.core import notes
import sympy as sp
import plotly.graph_objects as go

## Let's define our equations.
Note a lot of this work is from Claude so I'm not 100% certain of this... :^/

I've been trying to parse Compliant Mechanisms by Larry I. Howell but man is it dense.


Luckily I think it just boils down to a couple of important equations. Namely:
* Displacement of cantilever beam w/ Concentrated load at the free end
    * δ = (F * L^3 * cos(φ)) / (N * 3 * E * I)
    * δ: Displacement
    * F: Force
    * L: Length
    * E: Young's
    * I: Second moment of area which is I = W * H^3 / 12 for rectangular cross section
    * N: # of beams (assuming pairs here)
    * φ: Angle between beam & vertical (0 degrees for parallel beams forming rectangle as starting point)
* Maximum displacement
    * σ_max = (M_max * c) / I 
    * σ_max = (F * sin(θ) * L * c) / I
    * M_max: the bending moment (for a cantilever beam with concentrated load this comes to M_max = F * L)
        * M_max = F_perp * L = F * sin(θ) * L
        * θ: the angle between beam & horizontal (90 degrees for rectangle as starting point)
        * θ = 90 - φ
    * c: the distance from the neutral axis to the outer fibers (c = h/2 for a rectangular cross-section)
    * I: the second moment of area (I = bh^3/12 for a rectangular cross-section)
    * setting σ_max equal to the yield strength of the material and solving for the maximum bending moment (M_max), you can calculate the corresponding maximum displacement that would cause yielding.
TODO: find equations for y & z deflection (which we can minimize)


* Assumptions are based on Euler-bernoulli beam theory:
    * length >> cross-sectional dimensions
    * deflection << beam length
    * Pure bending deformation (no shear deformation)
* To keep these at bay:
    * H < ~0.2 L
    * W < ~0.2 L
    * δmax < 0.1 L


In [3]:
def calculate_displacement(
    force, # Newtons
    num_bridges, # unit-less
    bridge_length, # meters
    bridge_width, # meters
    bridge_height, # meters
    angle_from_vert, # angle
    # Default 69 GPa is for Aluminum 6061 https://en.wikipedia.org/wiki/6061_aluminium_alloy
    young_modulus=6.9e10,  # Pascals (N / m^2) 
):
    """
    δ = (F * L^3 * cos(φ)) / (N * 3 * E * I)
    * δ: Displacement
    * F: Force
    * L: Length
    * E: Young's Modulus
    * W: Width
    * H: Height
    * I: Second moment of area which is I = W * H^3 / 12 for rectangular cross section
    * N: # of beams (assuming pairs here)
    * φ: Angle between beam & vertical (0 degrees for parallel beams forming rectangle as starting point)
    """
    d, f, l, n, p, k, I, E, w, h = sp.symbols("d f l n p k I E w h")

    # Second moment of inertia for beam cross section
    I = w * (h**3) / 12
    # Stiffness
    k = (3 * E * I) / (l**3 * sp.cos(p))

    d = f / (n * k)

    equation = sp.Eq(d, f / k)
    return sp.solve(equation.subs({
        f: force,
        n: num_bridges,
        l: bridge_length,
        w: bridge_width,
        h: bridge_height,
        p: angle_from_vert,
        E: young_modulus,
    }), d)[0]